## 3D Pass Networks wit Python & Cinema4d
make sure to download Sergio's repository at https://github.com/Friends-of-Tracking-Data-FoTD/passing-networks-in-python


replace 'Demo Eventing.ipynb' with this notebook 

In [1]:
# Make sure that the path is the root of the project (it can be checked with '%pwd')
%cd ..

/Users/DML/OneDrive/Soccer Python/passing-networks-in-python-master


In [2]:
import pandas as pd
from pandas.io.json import json_normalize
from utils import read_json

In [3]:
lineups_path = "open-data-master/data/lineups/{0}.json"
events_path = "open-data-master/data/events/{0}.json"

team_name = "Spain"
match_id = 7576

### Step 1: Read data

In [5]:
lineups = read_json(lineups_path.format(match_id))
names_dict = {player["player_name"]: player["player_nickname"]
              for team in lineups for player in team["lineup"]}

names_dict

{'André Miguel Valente Silva': 'André Silva',
 'Andrés Iniesta Luján': 'Andrés Iniesta',
 'Bernardo Mota Veiga de Carvalho e Silva': 'Bernardo Silva',
 'Bruno Miguel Borges Fernandes': 'Bruno Fernandes',
 'Cristiano Ronaldo dos Santos Aveiro': 'Cristiano Ronaldo',
 'Cédric Ricardo Alves Soares': 'Cédric Soares',
 'David Josué Jiménez Silva': 'David Silva',
 'David de Gea Quintana': 'David de Gea',
 'Diego da Silva Costa': 'Diego Costa',
 'Francisco Román Alarcón Suárez': 'Isco',
 'Gerard Piqué Bernabéu': 'Gerard Piqué',
 'Gonçalo Manuel Ganchinho Guedes': 'Gonçalo Guedes',
 'Iago Aspas Juncal': 'Iago Aspas',
 'Jordi Alba Ramos': 'Jordi Alba',
 'Jorge Resurrección Merodio': 'Koke',
 'José Ignacio Fernández Iglesias': 'Nacho',
 'José Miguel da Rocha Fonte': 'José Fonte',
 'João Filipe Iria Santos Moutinho': 'João Moutinho',
 'João Mário Naval da Costa Eduardo': 'João Mário',
 'Kléper Laveran Lima Ferreira': 'Pepe',
 'Lucas Vázquez Iglesias': 'Lucas Vázquez',
 'Raphaël Adelino José Guerre

In [6]:
events = read_json(events_path.format(match_id))
df_events = json_normalize(events, sep="_").assign(match_id=match_id)

### Step 2: Compute max. minutes

In [7]:
first_red_card_minute = df_events[df_events.foul_committed_card_name.isin(["Second Yellow", "Red Card"])].minute.min()
first_substitution_minute = df_events[df_events.type_name == "Substitution"].minute.min()
max_minute = df_events.minute.max()

num_minutes = min(first_substitution_minute, first_red_card_minute, max_minute)
num_minutes

67

### Step 3: Set text information

In [8]:
plot_name = "statsbomb_match{0}_{1}".format(match_id, team_name)

opponent_team = [x for x in df_events.team_name.unique() if x != team_name][0]
plot_title ="{0}'s passing network against {1} (StatsBomb eventing data)".format(team_name, opponent_team)

plot_legend = "Location: pass origin\nSize: number of passes\nColor: number of passes"

### Step 4: Prepare data

In [9]:
def _statsbomb_to_point(location, max_width=120, max_height=80):
    '''
    Convert a point's coordinates from a StatsBomb's range to 0-1 range.
    '''
    return location[0] / max_width, 1-(location[1] / max_height)

In [10]:
df_passes = df_events[(df_events.type_name == "Pass") &
                      (df_events.pass_outcome_name.isna()) &
                      (df_events.team_name == team_name) &
                      (df_events.minute < num_minutes)].copy()

# If available, use player's nickname instead of full name to optimize space in plot
df_passes["pass_recipient_name"] = df_passes.pass_recipient_name.apply(lambda x: names_dict[x] if names_dict[x] else x)
df_passes["player_name"] = df_passes.player_name.apply(lambda x: names_dict[x] if names_dict[x] else x)

df_passes.head()
# df_passes.to_csv('passes.csv')

,ball_receipt_outcome_id,ball_receipt_outcome_name,ball_recovery_recovery_failure,carry_end_location,clearance_aerial_won,counterpress,dribble_outcome_id,dribble_outcome_name,dribble_overrun,duel_outcome_id,...,substitution_replacement_name,tactics_formation,tactics_lineup,team_id,team_name,timestamp,type_id,type_name,under_pressure,match_id
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,772,Spain,00:00:00.480,30,Pass,True,7576
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,772,Spain,00:00:01.173,30,Pass,True,7576
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,772,Spain,00:00:03.600,30,Pass,NaN,7576
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,772,Spain,00:00:04.413,30,Pass,True,7576
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,772,Spain,00:00:07.253,30,Pass,NaN,7576


In [11]:
df_passes["origin_pos_x"] = df_passes.location.apply(lambda x: _statsbomb_to_point(x)[0])
df_passes["origin_pos_y"] = df_passes.location.apply(lambda x: _statsbomb_to_point(x)[1])
player_position = df_passes.groupby("player_name").agg({"origin_pos_x": "median", "origin_pos_y": "median"})

player_position

,origin_pos_x,origin_pos_y
player_name,,
Andrés Iniesta,0.566667,0.78750
David Silva,0.716667,0.26250
David de Gea,0.079167,0.51875
Diego Costa,0.716667,0.72500
Gerard Piqué,0.366667,0.27500
Isco,0.591667,0.83750
Jordi Alba,0.541667,0.95000
Koke,0.562500,0.22500
Nacho,0.579167,0.02500


In [12]:
player_pass_count = df_passes.groupby("player_name").size().to_frame("num_passes")
player_pass_value = df_passes.groupby("player_name").size().to_frame("pass_value")

player_pass_count

,num_passes
player_name,
Andrés Iniesta,61
David Silva,33
David de Gea,10
Diego Costa,15
Gerard Piqué,51
Isco,71
Jordi Alba,65
Koke,50
Nacho,42


In [13]:
player_info = pd.concat([player_position, player_pass_count], axis=1, join_axes=[player_position.index])
player_info.to_csv('player_info.csv')
player_info

,origin_pos_x,origin_pos_y,num_passes
player_name,,,
Andrés Iniesta,0.566667,0.78750,61
David Silva,0.716667,0.26250,33
David de Gea,0.079167,0.51875,10
Diego Costa,0.716667,0.72500,15
Gerard Piqué,0.366667,0.27500,51
Isco,0.591667,0.83750,71
Jordi Alba,0.541667,0.95000,65
Koke,0.562500,0.22500,50
Nacho,0.579167,0.02500,42


In [14]:
df_passes_low = df_passes[(df_passes.pass_height_name == "Ground Pass") | (df_passes.pass_height_name == "Low Pass")]
df_passes_high = df_passes[(df_passes.pass_height_name == "High Pass")]

In [15]:
df_passes["pair_key"] = df_passes.apply(lambda x: "_".join(sorted([x["player_name"], x["pass_recipient_name"]])), axis=1)

pair_pass_count = df_passes.groupby("pair_key").size().to_frame("num_passes")
pair_pass_value = df_passes.groupby("pair_key").size().to_frame("pass_value")

pair_pass_value['origin_pos_player_x'] = 0
pair_pass_value['origin_pos_player_y'] = 0
pair_pass_value['origin_pos_recipe_x'] = 0
pair_pass_value['origin_pos_recipe_y'] = 0

for index, row in pair_pass_value.iterrows():
    pair_pass_value.loc[index,'origin_pos_player_x'] = player_info['origin_pos_x'][index.rsplit('_',1)[0]] 
    pair_pass_value.loc[index,'origin_pos_player_y'] = player_info['origin_pos_y'][index.rsplit('_',1)[0]] 
    pair_pass_value.loc[index,'origin_pos_recipe_x'] = player_info['origin_pos_x'][index.rsplit('_',1)[1]] 
    pair_pass_value.loc[index,'origin_pos_recipe_y'] = player_info['origin_pos_y'][index.rsplit('_',1)[1]]
    #print(index.rsplit('_',1)[0])
    #print(index.rsplit('_',1)[1])
pair_pass_value.to_csv('pair_pass_value.csv')    
pair_pass_value

,pass_value,origin_pos_player_x,origin_pos_player_y,origin_pos_recipe_x,origin_pos_recipe_y
pair_key,,,,,
Andrés Iniesta_David Silva,5,0.566667,0.78750,0.716667,0.2625
Andrés Iniesta_Diego Costa,7,0.566667,0.78750,0.716667,0.7250
Andrés Iniesta_Gerard Piqué,6,0.566667,0.78750,0.366667,0.2750
Andrés Iniesta_Isco,28,0.566667,0.78750,0.591667,0.8375
Andrés Iniesta_Jordi Alba,27,0.566667,0.78750,0.541667,0.9500
Andrés Iniesta_Koke,7,0.566667,0.78750,0.562500,0.2250
Andrés Iniesta_Nacho,1,0.566667,0.78750,0.579167,0.0250
Andrés Iniesta_Sergio Busquets,9,0.566667,0.78750,0.466667,0.4875
Andrés Iniesta_Sergio Ramos,38,0.566667,0.78750,0.425000,0.7000


In [16]:
df_passes_low["pair_key"] = df_passes_low.apply(lambda x: "_".join(sorted([x["player_name"], x["pass_recipient_name"]])), axis=1)

pair_pass_count_low = df_passes_low.groupby("pair_key").size().to_frame("num_passes")
pair_pass_value_low = df_passes_low.groupby("pair_key").size().to_frame("pass_value")

pair_pass_value_low['origin_pos_player_x'] = 0
pair_pass_value_low['origin_pos_player_y'] = 0
pair_pass_value_low['origin_pos_recipe_x'] = 0
pair_pass_value_low['origin_pos_recipe_y'] = 0

for index, row in pair_pass_value_low.iterrows():
    pair_pass_value_low.loc[index,'origin_pos_player_x'] = player_info['origin_pos_x'][index.rsplit('_',1)[0]] 
    pair_pass_value_low.loc[index,'origin_pos_player_y'] = player_info['origin_pos_y'][index.rsplit('_',1)[0]] 
    pair_pass_value_low.loc[index,'origin_pos_recipe_x'] = player_info['origin_pos_x'][index.rsplit('_',1)[1]] 
    pair_pass_value_low.loc[index,'origin_pos_recipe_y'] = player_info['origin_pos_y'][index.rsplit('_',1)[1]]
    #print(index.rsplit('_',1)[0])
    #print(index.rsplit('_',1)[1])
pair_pass_value_low = pair_pass_value_low.add_suffix('_low')
  
pair_pass_value_low

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,pass_value_low,origin_pos_player_x_low,origin_pos_player_y_low,origin_pos_recipe_x_low,origin_pos_recipe_y_low
pair_key,,,,,
Andrés Iniesta_David Silva,5,0.566667,0.78750,0.716667,0.2625
Andrés Iniesta_Diego Costa,7,0.566667,0.78750,0.716667,0.7250
Andrés Iniesta_Gerard Piqué,6,0.566667,0.78750,0.366667,0.2750
Andrés Iniesta_Isco,28,0.566667,0.78750,0.591667,0.8375
Andrés Iniesta_Jordi Alba,27,0.566667,0.78750,0.541667,0.9500
Andrés Iniesta_Koke,7,0.566667,0.78750,0.562500,0.2250
Andrés Iniesta_Sergio Busquets,9,0.566667,0.78750,0.466667,0.4875
Andrés Iniesta_Sergio Ramos,38,0.566667,0.78750,0.425000,0.7000
David Silva_Diego Costa,3,0.716667,0.26250,0.716667,0.7250


In [17]:
df_passes_high["pair_key"] = df_passes_high.apply(lambda x: "_".join(sorted([x["player_name"], x["pass_recipient_name"]])), axis=1)

pair_pass_count_high = df_passes_high.groupby("pair_key").size().to_frame("num_passes")
pair_pass_value_high = df_passes_high.groupby("pair_key").size().to_frame("pass_value")

pair_pass_value_high['origin_pos_player_x'] = 0
pair_pass_value_high['origin_pos_player_y'] = 0
pair_pass_value_high['origin_pos_recipe_x'] = 0
pair_pass_value_high['origin_pos_recipe_y'] = 0

for index, row in pair_pass_value_high.iterrows():
    pair_pass_value_high.loc[index,'origin_pos_player_x'] = player_info['origin_pos_x'][index.rsplit('_',1)[0]] 
    pair_pass_value_high.loc[index,'origin_pos_player_y'] = player_info['origin_pos_y'][index.rsplit('_',1)[0]] 
    pair_pass_value_high.loc[index,'origin_pos_recipe_x'] = player_info['origin_pos_x'][index.rsplit('_',1)[1]] 
    pair_pass_value_high.loc[index,'origin_pos_recipe_y'] = player_info['origin_pos_y'][index.rsplit('_',1)[1]]
    #print(index.rsplit('_',1)[0])
    #print(index.rsplit('_',1)[1])
    
pair_pass_value_high = pair_pass_value_high.add_suffix('_high')
# pair_pass_value_high = pair_pass_value_high.to_csv('pair_pass_value_high.csv')    
pair_pass_value_high

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,pass_value_high,origin_pos_player_x_high,origin_pos_player_y_high,origin_pos_recipe_x_high,origin_pos_recipe_y_high
pair_key,,,,,
Andrés Iniesta_Nacho,1,0.566667,0.78750,0.579167,0.0250
David Silva_Sergio Busquets,2,0.716667,0.26250,0.466667,0.4875
David de Gea_Isco,1,0.079167,0.51875,0.591667,0.8375
David de Gea_Nacho,2,0.079167,0.51875,0.579167,0.0250
Diego Costa_Isco,2,0.716667,0.72500,0.591667,0.8375
Diego Costa_Jordi Alba,1,0.716667,0.72500,0.541667,0.9500
Gerard Piqué_Jordi Alba,1,0.366667,0.27500,0.541667,0.9500
Gerard Piqué_Sergio Busquets,1,0.366667,0.27500,0.466667,0.4875
Isco_Nacho,2,0.591667,0.83750,0.579167,0.0250


In [18]:
joined = pd.concat([pair_pass_value, pair_pass_value_low['pass_value_low'].reindex(pair_pass_value.index)], axis=1)
joined = pd.concat([joined, pair_pass_value_high['pass_value_high'].reindex(joined.index)], axis=1)
joined = joined.fillna(0)

joined = joined[['pass_value', 'pass_value_low','pass_value_high', 'origin_pos_player_x' , 'origin_pos_player_y', 'origin_pos_recipe_x' , 'origin_pos_recipe_y' ]]
joined.to_csv('pair_pass_values.csv')  
joined

,pass_value,pass_value_low,pass_value_high,origin_pos_player_x,origin_pos_player_y,origin_pos_recipe_x,origin_pos_recipe_y
pair_key,,,,,,,
Andrés Iniesta_David Silva,5,5.0,0.0,0.566667,0.78750,0.716667,0.2625
Andrés Iniesta_Diego Costa,7,7.0,0.0,0.566667,0.78750,0.716667,0.7250
Andrés Iniesta_Gerard Piqué,6,6.0,0.0,0.566667,0.78750,0.366667,0.2750
Andrés Iniesta_Isco,28,28.0,0.0,0.566667,0.78750,0.591667,0.8375
Andrés Iniesta_Jordi Alba,27,27.0,0.0,0.566667,0.78750,0.541667,0.9500
Andrés Iniesta_Koke,7,7.0,0.0,0.566667,0.78750,0.562500,0.2250
Andrés Iniesta_Nacho,1,0.0,1.0,0.566667,0.78750,0.579167,0.0250
Andrés Iniesta_Sergio Busquets,9,9.0,0.0,0.566667,0.78750,0.466667,0.4875
Andrés Iniesta_Sergio Ramos,38,38.0,0.0,0.566667,0.78750,0.425000,0.7000
